In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0kja2flg
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0kja2flg
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=5fad7b874b262add925388e4f9e0fd4c0fceee3126a17c86109d9df2b7a9fca8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xhgq_zm9/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


ADDING TWO LARGE VECTORS

In [6]:
%%cu
#include <iostream>
#include <fstream>
#include <cstdlib>
#include <cassert>
using namespace std;
__global__ void vectorAdd(int *a, int *b, int *result, int N)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N)
        result[tid] = a[tid] + b[tid];
}
void init_array(int *a, int N)
{
    for (int i = 0; i < N; i++)
    {
        a[i] = rand() % 100;
    }
}

void verify_solution(int *a, int *b, int *result, int N)
{
    for (int i = 0; i < N; i++)
    {
        assert(a[i] + b[i] == result[i]);
    }
}
int main()
{
    cout << "Here";
  //  int N = 1 << 20;
    int N = 100;
    size_t bytes = N * sizeof(int);
    int *a, *b, *result;
    cudaMallocManaged(&a, bytes);
    cudaMallocManaged(&b, bytes);
    cudaMallocManaged(&result, bytes);
    init_array(a, N);
    init_array(b, N);
    cout << "Vector a : \n";
    for (int i = 0; i < N; i++)
    {
        cout << a[i] << " ";
    }
    cout << "\n\nVector b : \n";
    for (int i = 0; i < N; i++)
    {
        cout << b[i] << " ";
    }
     cudaEvent_t start, end;
     cudaEventCreate(&start);
     cudaEventCreate(&end);
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    vectorAdd<<<blocks, threads>>>(a, b, result, N);
      cudaEventRecord(end);
     cudaEventSynchronize(end);
    float time = 0.0;
     cudaEventElapsedTime(&time, start, end);
     cudaDeviceSynchronize();
    cout << "\n\nresult : \n";
    for (int i = 0; i < N; i++)
    {
        cout << result[i] << " ";
    }
    verify_solution(a, b, result, N);
    cout << "Program completed successfully!" << endl;
    cout << "Time Elapsed : " << time;
    return 0;
}



HereVector a : 
83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 36 5 46 29 13 57 24 95 82 45 14 67 34 64 43 50 87 8 76 78 88 84 3 51 54 99 32 60 76 68 39 12 26 86 94 39 

Vector b : 
95 70 34 78 67 1 97 2 17 92 52 56 1 80 86 41 65 89 44 19 40 29 31 17 97 71 81 75 9 27 67 56 97 53 86 65 6 83 19 24 28 71 32 29 3 19 70 68 8 15 40 49 96 23 18 45 46 51 21 55 79 88 64 28 41 50 93 0 34 64 24 14 87 56 43 91 27 65 59 36 32 51 37 28 75 7 74 21 58 95 29 37 35 93 18 28 43 11 28 29 

result : 
178 156 111 93 160 36 183 94 66 113 114 83 91 139 149 67 105 115 116 55 51 97 98 46 179 101 143 98 76 62 96 58 119 111 155 132 99 139 30 66 57 144 53 48 87 56 168 92 23 85 53 75 187 103 74 118 108 121 117 136 84 113 148 55 77 55 139 29 47 121 48 109 169 101 57 158 61 129 102 86 119 59 113 106 163 91 77 72 112 194 61 97 111 161 57 40 69 97 122 68 Program completed succes

Multiply Vector and Matrix

In [7]:
%%cu
#include <iostream>
using namespace std;
__global__ void matrixVector(int *vec, int *mat, int *result, int n, int m)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    if (tid <= n)
    {
        for (int i = 0; i < n; i++)
        {
            sum += vec[i] * mat[(i * m) + tid];
        }
        result[tid] = sum;
    }
}
void init_array(int *a, int n)
{
    for (int i = 0; i < n; i++)
        a[i] = rand() % n + 1;
}
void init_matrix(int *a, int n, int m)
{
    for (int i = 0; i < n; i++)
    {
        for (int j = 0; j < m; j++)
        {
            a[i * m + j] = rand() % n + 1;
        }
    }
}
void print_array(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        cout << " " << a[i];
    }
    cout << endl;
}
void print_matrix(int *a, int n, int m)
{
    for (int i = 0; i < n; i++)
    {
        for (int j = 0; j < m; j++)
            cout << " " << a[i * m + j];
        cout << endl;
    }
}
int main()
{
    int *a, *b, *c;
    int *a_dev, *b_dev, *c_dev;
    int n = 3;
    int m = 4;
    a = new int[n];
    b = new int[n * m];
    c = new int[m];
    init_array(a, n);
    init_matrix(b, n, m);
    cout << "Initial vector : " << endl;
    print_array(a, n);
    cout << "Initial matrix : " << endl;
    print_matrix(b, n, m);
    cout << endl;
    cudaMalloc(&a_dev, sizeof(int) * n);
    cudaMalloc(&b_dev, sizeof(int) * n * m);
    cudaMalloc(&c_dev, sizeof(int) * m);
    cudaMemcpy(a_dev, a, sizeof(int) * n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, sizeof(int) * n * m, cudaMemcpyHostToDevice);
    matrixVector<<<m / 256 + 1, 256>>>(a_dev, b_dev, c_dev, n, m);
    cudaMemcpy(c, c_dev, sizeof(int) * m, cudaMemcpyDeviceToHost);
    cout << "Results : " << endl;
    print_array(c, m);
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    delete[] a;
    delete[] b;
    delete[] c;
    return 0;
}


Initial vector : 
 2 2 1
Initial matrix : 
 2 3 2 2
 1 1 2 3
 2 3 2 3

Results : 
 8 11 10 13



Multiply two N × N arrays using n2
processors

In [8]:
%%cu
#include<iostream>
#include<cstdlib>
#include<cmath>
using namespace std;

//Matrix multiplication Cuda 
__global__ void matrixMultiplication(int *a, int *b, int *c, int n)
{
    int row=threadIdx.y+blockDim.y*blockIdx.y;
    int col=threadIdx.x+blockDim.x*blockIdx.x;
    int sum=0;
   
    if(row<n && col<n)
    for(int j=0;j<n;j++)
    {
        sum=sum+a[row*n+j]*b[j*n+col];
    }
    
    c[n*row+col]=sum;
}
int main()
{
    int *a,*b,*c;
    int *a_dev,*b_dev,*c_dev;
    int n=3;
    
    a=new int[n*n];
    b=new int[n*n];
    c=new int[n*n];
    int *d=new int[n*n];
    int size=n*n*sizeof(int);
    cudaMalloc(&a_dev,size);
    cudaMalloc(&b_dev,size);
    cudaMalloc(&c_dev,size);
    
    //Array initialization
    for(int i=0;i<n*n;i++)
    {
          a[i]=rand()%n;
        b[i]=rand()%n;
       // d[i]=a[i]+b[i];
    }
    
    // printing a 
    cout << "a : \n";
    for(int i=0; i<n; i++)
    {
      for(int j=0;j<n;j++)
          cout << a[i*n + j] << " ";
          cout << endl;
    }
    
    // printing b 
    cout << "b : \n";
    for(int i=0; i<n; i++)
    {
      for(int j=0;j<n;j++)
          cout << b[i*n + j] << " ";
          cout << endl;
    }

    cudaEvent_t start,end;
    
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    
    cudaMemcpy(a_dev,a,size,cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev,b,size,cudaMemcpyHostToDevice);
    
    
    dim3 threadsPerBlock(n, n);
    dim3 blocksPerGrid(1, 1);
    
    if(n*n>512){
        threadsPerBlock.x=512;
        threadsPerBlock.y=512;
        blocksPerGrid.x=ceil((double)n/(double)threadsPerBlock.x);
        blocksPerGrid.y=ceil((double)n/(double)threadsPerBlock.y);
    }
    //GPU Multiplication
    cudaEventRecord(start);
    matrixMultiplication<<<blocksPerGrid,threadsPerBlock>>>(a_dev,b_dev,c_dev,n);
    
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    
    float time=0.0;
    cudaEventElapsedTime(&time,start,end);
    
    cudaMemcpy(c,c_dev,size,cudaMemcpyDeviceToHost);
    
    
    //CPU matrix multiplication
    int sum=0;
    for(int row=0;row<n;row++)
    {
        for(int col=0;col<n;col++)
        {
           sum=0;
           for(int k=0;k<n;k++)
              sum=sum+a[row*n+k]*b[k*n+col];  
            d[row*n+col]=sum;
        }
      
    }
    int error=0;
    for(int i=0;i<n*n;i++){
        error+=d[i]-c[i];
        // cout<<" gpu "<<c[i]<<" CPU "<<d[i]<<endl;
    }
    
    // printing c 
    cout << "c : \n";
    for(int i=0; i<n; i++)
    {
      for(int j=0;j<n;j++)
          cout << c[i*n + j] << " ";
          cout << endl;
    }
 
    cout<<"Error : "<<error;
    cout<<"\nTime Elapsed:  "<<time;
    
    return 0;
}

a : 
1 0 2 
1 0 2 
2 2 0 
b : 
1 1 1 
0 1 1 
1 1 0 
c : 
3 3 1 
3 3 1 
2 4 4 
Error : 0
Time Elapsed:  0.113856
